In [1]:
# 1. Install required libraries
!pip install -q segment-anything-py
!pip install -q scikit-image

# 2. Create directories for models and output
!mkdir -p /content/sam_weights
!mkdir -p /content/output

# 3. Download the Segment-Anything-Model (SAM) weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth -O /content/sam_weights/sam_vit_l_0b3195.pth

print("✅ Environment Setup Complete!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.1 MB/s eta 0:00:00
✅ Environment Setup Complete!


In [12]:
import torch
import cv2
import numpy as np
from PIL import Image
import requests
import io
import base64
from skimage import io as skio
from segment_anything import sam_model_registry, SamPredictor
from IPython.display import display

# ===============================
# Load SAM Model
# ===============================
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
try:
    sam_l = sam_model_registry['vit_l'](checkpoint='/content/sam_weights/sam_vit_l_0b3195.pth').to(device=DEVICE)
    print("✅ SAM Model Loaded")
except Exception as e:
    print(f"❌ Error loading SAM model:  {e}")

# ===============================
# Helper Functions
# ===============================
def call_sd_api(api_url, endpoint, payload):
    try:
        response = requests.post(url=f'{api_url}/{endpoint}', json=payload, timeout=600)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ API request failed:  {e}")
        return None

def decode_and_resize(image_data, width, height):
    image = Image.open(io.BytesIO(base64.b64decode(image_data.split(",", 1)[0])))
    image = image.resize((width, height))
    return np.array(image)

# ===============================
# Image Processing Classes
# ===============================
class BackRemover:
    def __init__(self, image):
        self.image = image

    def image_preprocessor(self):
        predictor = SamPredictor(sam_l)
        predictor.set_image(self.image)
        h, w, _ = self.image.shape
        input_box = np.array([w * 0.05, h * 0.05, w * 0.95, h * 0.95])
        masks, _, _ = predictor.predict(box=input_box, multimask_output=False)
        mask = masks[0]
        alpha_channel = np.where(mask, 255, 0).astype(np.uint8)
        bgr_image = cv2.cvtColor(self.image, cv2.COLOR_RGB2BGR)
        bgra_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2BGRA)
        bgra_image[:, :, 3] = alpha_channel
        x, y, w, h = cv2.boundingRect(alpha_channel)
        return bgra_image[y:y+h, x:x+w]

class AdGenerator:
    def __init__(self, product_image_path, api_url):
        self.product_image_path = product_image_path
        self.api_url = api_url
        self.transparent_product = self._prepare_product_image()

    def _prepare_product_image(self):
        print("️Preparing product image...")
        image = skio.imread(self.product_image_path)
        if image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        remover = BackRemover(image)
        return remover.image_preprocessor()

    def generate(self, background_prompt, final_blend_prompt, negative_prompt, width=910, height=512):
        if self.transparent_product is None: return None

        print("Step 1: Generating background...")
        payload = {
            "prompt": background_prompt,
            "negative_prompt": negative_prompt,
            "steps": 25,
            "width": width,
            "height": height,
            "seed": -1
        }
        r = call_sd_api(self.api_url, 'sdapi/v1/txt2img', payload)
        if not r or 'images' not in r: return None
        background = decode_and_resize(r['images'][0], width, height)
        background = cv2.cvtColor(background, cv2.COLOR_RGB2BGR)

        print("Step 2: Compositing product with enhancements...")
        max_prod_width, max_prod_height = int(width * 0.7), int(height * 0.8)
        h_orig, w_orig, _ = self.transparent_product.shape
        ratio = min(max_prod_width / w_orig, max_prod_height / h_orig)
        new_w, new_h = int(w_orig * ratio), int(h_orig * ratio)
        product_resized = cv2.resize(self.transparent_product, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

        product_resized[:, :, :3] = cv2.addWeighted(
            product_resized[:, :, :3], 0.95,
            np.full_like(product_resized[:, :, :3], (10, 20, 0), dtype=np.uint8), 0.05, 0
        )

        h_prod, w_prod, _ = product_resized.shape
        h_bg, w_bg, _ = background.shape
        y_offset, x_offset = (h_bg - h_prod) // 2, (w_bg - w_prod) // 2
        overlay = background.copy()

        # --- SHADOW FIX: Made shadow softer and more transparent ---
        shadow_shape = np.zeros_like(overlay)
        shadow_shape[y_offset:y_offset+h_prod, x_offset:x_offset+w_prod] = (50, 50, 50)
        blurred_shadow_shape = cv2.GaussianBlur(shadow_shape, (75, 75), 0)

        shadow_offset_x, shadow_offset_y = 15, 15
        shadow_y_start, shadow_x_start = y_offset + shadow_offset_y, x_offset + shadow_offset_x
        shadow_y_end, shadow_x_end = min(shadow_y_start + h_prod, h_bg), min(shadow_x_start + w_prod, w_bg)
        shadow_roi = overlay[shadow_y_start:shadow_y_end, shadow_x_start:shadow_x_end]
        blurred_shadow_roi = blurred_shadow_shape[y_offset:y_offset+(shadow_y_end - shadow_y_start),
                                                  x_offset:x_offset+(shadow_x_end - shadow_x_start)]

        alpha_mask = cv2.resize(product_resized[:, :, 3], (shadow_roi.shape[1], shadow_roi.shape[0])) / 255.0
        alpha_mask = np.expand_dims(alpha_mask, axis=2)

        overlay[shadow_y_start:shadow_y_end, shadow_x_start:shadow_x_end] = \
            (1 - alpha_mask * 0.3) * shadow_roi + (alpha_mask * 0.3) * blurred_shadow_roi
        # --- END OF SHADOW FIX ---

        product_mask = product_resized[:, :, 3] / 255.0
        for c in range(3):
            overlay[y_offset:y_offset+h_prod, x_offset:x_offset+w_prod, c] = (
                (1.0 - product_mask) * overlay[y_offset:y_offset+h_prod, x_offset:x_offset+w_prod, c] +
                product_mask * product_resized[:, :, c]
            )

        print("Step 3: Final polish using img2img...")
        outpaint_mask = np.zeros((h_bg, w_bg), dtype=np.uint8)
        outpaint_mask[y_offset:y_offset+h_prod, x_offset:x_offset+w_prod] = 255
        _, buffer_overlay = cv2.imencode('.png', overlay)
        overlay_b64 = base64.b64encode(buffer_overlay).decode('utf-8')
        _, buffer_mask = cv2.imencode('.png', outpaint_mask)
        mask_b64 = base64.b64encode(buffer_mask).decode('utf-8')

        payload = {
            "init_images": [overlay_b64],
            "mask": mask_b64,
            "inpainting_mask_invert": 1,
            "prompt": final_blend_prompt,
            "negative_prompt": negative_prompt,
            "steps": 50,
            "denoising_strength": 0.35,
            "width": width,
            "height": height,
            "seed": -1,
            "inpainting_fill": 1
        }
        r = call_sd_api(self.api_url, 'sdapi/v1/img2img', payload)
        if not r or 'images' not in r: return None
        final_image_np = decode_and_resize(r['images'][0], width, height)
        return Image.fromarray(final_image_np)

# ===============================
# SCRIPT ENTRY POINT
# ===============================
if __name__ == '__main__':
    api_url = "https://c3a8f831a5dfb3e739.gradio.live"
    product_image_path = "/content/51zuwnYnMWL._SL1000_.jpg"
    output_path = "/content/output/generated_ad_final.png"

    background_prompt = (
        "A rustic wooden kitchen counter with morning sunlight streaming in, a fresh jar of tomato sauce at the center, "
        "surrounded by ripe tomatoes, garlic cloves, basil leaves, and a wooden spoon. The background includes a vintage Italian kitchen setup. "
        "Professional food photography, high contrast, shallow depth of field, realistic textures, 8k, ultra high detail, warm tones"
    )

    final_blend_prompt = (
        background_prompt +
        ", seamless blend between product and background, natural shadow under the bottle, realistic light reflection on the bottle, photo-realistic, smooth textures"
    )

    negative_prompt = "blurry, dark, grainy, text, watermark, people, hands, clutter, modern, plastic, cartoon, anime"

    if "gradio.live" not in api_url and "127.0.0.1" not in api_url:
        print("❌ ERROR: Please set a valid public `api_url`.")
    else:
        print("🚀 Starting Final Hackathon Ad Generation Pipeline...")
        generator = AdGenerator(product_image_path, api_url)
        final_ad_image = generator.generate(background_prompt, final_blend_prompt, negative_prompt)

        if final_ad_image:
            final_ad_image.save(output_path)
            print(f"✅ Success! Image saved to  {output_path}")
            display(final_ad_image)
        else:
            print("❌ Ad generation failed.")


✅ SAM Model Loaded
🚀 Starting Final Hackathon Ad Generation Pipeline...
️Preparing product image...
Step 1: Generating background...
❌ API request failed:  404 Client Error: Not Found for url: https://c3a8f831a5dfb3e739.gradio.live/sdapi/v1/txt2img
❌ Ad generation failed.
